In [82]:
import pandas as pd 
import string 
from snowballstemmer import TurkishStemmer

import re
import numpy as np

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

from wordcloud import WordCloud


import tensorflow as tf
from keras._tf_keras.keras.preprocessing.sequence import pad_sequences
from keras._tf_keras.keras.layers import Embedding , LSTM ,Dense,Dropout,Bidirectional,Flatten
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.preprocessing.text import one_hot,Tokenizer

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

# Veri Ön İşleme

In [66]:
#noktalama kaldırma
df = pd.read_csv("data.csv")
noktalamasiz= []
for i in df.text:
  son = ""
  for j in i:
    if j not in string.punctuation:
      son += j
  noktalamasiz.append(son)
df["Temiz"] = noktalamasiz

     

# Büyük Küçük Harfler

In [67]:

#büyük harfleri küçük harfe dönüştürme ve tokenize
kucukHarfveTokenize=[]
for text in df.Temiz:
  text=text.lower()
  text=text.split()
  kucukHarfveTokenize.append(text)

df["Temiz"] = kucukHarfveTokenize



# Stopwords Kaldırma

In [68]:
stopwords=[]
with open("stopWords.txt","r") as file :
    for satir in file:
        satir=satir.strip().lower()
        if len(satir)>1:
            stopwords.append(satir)
            
stopwordsuz= []
for text in df.Temiz:
  son =[]
  for i in text:
    if i not in stopwords:
      son.append(i)
  stopwordsuz.append(son)
df["Temiz"] = stopwordsuz

# Stemming 

In [69]:
turkStem=TurkishStemmer()
stemmed=[]
for text in df.text:
  stemmed.append(turkStem.stemWord(text)) 
df["Stemmed"] = stemmed



# Word2Vec

In [70]:
y=df["label"].values
X=[d.split() for d in df['Stemmed']]

In [74]:
DIM=100
model_w2v=Word2Vec(sentences=X, vector_size=DIM, window=10, min_count=1)

# FastText

In [73]:
from gensim.models import FastText
model_fast = FastText(sentences=X, vector_size=DIM, window=5, min_count=5)

In [75]:
X= df["Stemmed"]
y= df["label"]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
x_train = x_train.values
x_test = x_test.values

(1984,)
(1984,)
(496,)
(496,)


# Tokenizer

In [76]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)
X=tokenizer.texts_to_sequences(X)
vocab = tokenizer.word_index

In [77]:
def max_len(X):
  max = 0
  for i in X:
    if len(i)>max:
        max=len(i)
  return max

maxlen=max_len(X)
X=pad_sequences(X, padding="post",maxlen=maxlen)
vocab_size=len(tokenizer.word_index)+1 




In [80]:
def matrix(model):
    weight_matrix = np.zeros((10000,DIM))
    for word, i in vocab.items():
        weight_matrix[i] = model.wv[word] 

    return weight_matrix

embedding_vectors_w2v = matrix(model_w2v)
embedding_vectors_fast = matrix(model_fast)


# Model1 Embedding Bidirectional LSTM

In [91]:
model=Sequential()
model.add(Embedding(input_dim = 10000,#
                    output_dim = DIM,#
                    weights = [embedding_vectors_w2v],
                    input_length = maxlen,
                    trainable = False))
model.add(LSTM(100, dropout=0.2)) # lstm neuron size
model.add(Dense(1,activation='sigmoid')) # dense layer, activation function
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

model.summary()

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,000,000 (3.81 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,000,000 (3.81 MB)

In [93]:
history = model.fit(x_train, y_train, validation_split=0.2, epochs=10, batch_size=64)

Epoch 1/10


ValueError: Sequential model 'sequential_7' has already been configured to use input shape (64,). You cannot build it with input_shape (None,)